In [ ]:
pip install crewai ollama

In [ ]:
pip install crewai_tools

In [ ]:
pip install langchain-ollama

In [ ]:
pip install ollama

In [1]:
import os
os.environ["OPENAI_API_BASE"] = 'http://localhost:11434'
os.environ["OPENAI_MODEL_NAME"] = 'llama3.2:3b'  # Adjust based on the model you have
os.environ["OPENAI_API_KEY"] = ''  # No API key needed for Ollama


In [2]:
import os
from crewai import Agent, Task, Crew, Process
from langchain_ollama import ChatOllama
import time

# Set the environment variable to ensure no OpenAI key dependency
os.environ["OPENAI_API_KEY"] = "NA"  # Not required for Ollama
os.environ["OPENAI_API_BASE"] = 'http://localhost:11434'  # Ollama local endpoint

# Configure Ollama for the student and teacher agents
llm = ChatOllama(
    model="llama3.2:3b",  # The local Ollama model you're using
    base_url="http://localhost:11434"
)

# Define function to query the Ollama model using the correct Ollama API
def ask_ollama(prompt):
    try:
        response = llm.chat(prompt)
        return response['text']  # Extract the text response
    except Exception as e:
        print(f"Error querying Ollama: {e}")
        return "Error: Unable to query Ollama."

# Define the Student agent using crewAI with Ollama-based LLM
student = Agent(
    role="Statistics Student",
    goal="Learn statistics step by step by asking questions.",
    backstory="""You are a diligent student eager to learn statistics. 
    You start by asking easy questions and then progress to more advanced topics based on the responses you get.""",
    verbose=True,
    llm=llm,  # Using Ollama for the student agent's LLM
    allow_delegation=True
)

# Define the Teacher agent that uses Ollama to answer student questions
teacher = Agent(
    role="Statistics Teacher",
    goal="Explain statistical concepts to the student, from basic to advanced topics, step by step.",
    backstory="""You are a knowledgeable teacher of statistics, capable of explaining statistical concepts 
    from simple to complex in an easy-to-understand manner.""",
    verbose=True,
    llm=llm,  # Using Ollama for the teacher agent's LLM
    allow_delegation=True
)

# Define the student's task to ask the teacher questions freely
student_task = Task(
    description="The student will start by asking basic questions about statistics and progressively ask more advanced questions as the session continues.",
    expected_output="The student will ask progressively harder questions to improve understanding of statistics.",
    agent=student,
    async_execution=False  # Synchronous execution
)

# Define the teacher's task to respond to the student's questions using Ollama model
teacher_task = Task(
    description="The teacher will respond with detailed explanations of statistical concepts based on the student's questions.",
    expected_output="The teacher will provide clear and concise explanations of statistical concepts.",
    agent=teacher,
    context=[student_task],
    async_execution=False  # Synchronous execution
)

# Assemble the crew with agents, tasks, and a sequential process
my_crew = Crew(
    agents=[student, teacher],
    tasks=[student_task, teacher_task],
    process=Process.sequential,  # Tasks will be executed in sequence
    full_output=True,  # Output will include all task outputs
    verbose=True,  # Verbose logging for detailed execution info
)

# Kickoff the crew process
print("\n--- Starting the learning session ---")
crew_output = my_crew.kickoff()

# Start the learning session where the student asks questions in free will
student_progress = "basic"  # Initially, the student starts with basic questions
continue_learning = True
question_number = 1

while continue_learning:
    print(f"\n--- Student is preparing to ask question {question_number} ---")
    
    # The student generates a question dynamically based on their progress
    if student_progress == "basic":
        question = ask_ollama("Generate a basic question about statistics.")
    elif student_progress == "intermediate":
        question = ask_ollama("Generate an intermediate question about statistics.")
    else:
        question = ask_ollama("Generate an advanced question about statistics.")
    
    print(f"Student: {question.strip()}")

    # Teacher provides an answer using the Ollama model
    teacher_response = ask_ollama(f"The student asked: {question}. Provide a detailed explanation.")
    print(f"Teacher: {teacher_response}")

    # Decide whether the student should progress to more advanced topics or continue
    if question_number >= 3 and student_progress == "basic":
        student_progress = "intermediate"
        print("\n--- The student is now moving to intermediate level questions ---")
    elif question_number >= 6 and student_progress == "intermediate":
        student_progress = "advanced"
        print("\n--- The student is now moving to advanced level questions ---")
    elif question_number >= 9:
        # After 10 questions, end the session
        print("\n--- The student has asked 10 questions. Ending the session ---")
        continue_learning = False

    # Delay to simulate a more natural flow
    time.sleep(2)
    question_number += 1

# Access task output and print results
print("\n--- Task Outputs ---")
print(f"Raw Crew Output: {crew_output.raw}")

# If there's JSON output, print it
if crew_output.json_dict:
    print(f"JSON Output: {crew_output.json_dict}")

# Access task-specific outputs
for task_output in crew_output.tasks_output:
    print(f"Task Output Description: {task_output.description}")
    print(f"Task Raw Output: {task_output.raw}")

# Check token usage
print(f"Token Usage: {crew_output.token_usage}")



--- Starting the learning session ---
 [2024-09-26 00:35:21][DEBUG]: == Working Agent: Statistics Student
 [2024-09-26 00:35:21][INFO]: == Starting Task: The student will start by asking basic questions about statistics and progressively ask more advanced questions as the session continues.


> Entering new CrewAgentExecutor chain...
Let's start by asking a basic question to get a feel for statistics. I'll ask a simple question to my coworker, the Statistics Teacher.

Action: Ask question to coworker
Action Input: {'question': 'What is the meaning of the term "mean" in statistics?', 'context': 'I am new to statistics and need an explanation.', 'coworker': 'Statistics Teacher

> Entering new CrewAgentExecutor chain...
Final Answer: The term "mean" in statistics refers to a measure of central tendency that represents the average value of a dataset or a set of numbers. It's calculated by summing up all the values in the data set and then dividing by the total number of values. In other w

KeyboardInterrupt: 